In [1]:
from Q_agent import DQNAgent
from robot import Cam
from robot import Pepper
import thread
import time
from PIL import Image
import torch
from torchvision.transforms import Compose,ToTensor,Resize
Image_size=[240,240]

In [2]:
ip='192.168.1.166'
robot = Pepper.Pepper(ip=ip)
# cam = Cam.Cam()
Q_Agent = DQNAgent()

In [3]:
robot.perform_act('nod')

In [4]:
cam = Cam.Cam()

init camera


# RL execution

### settings

In [45]:
ep_file='RL_DATA/EP.data'
time_unit = 3
camera = Cam.fishcam
actions=robot.action_list
ep = int(open(ep_file).read())
time_stamp = 0
def trans(imgs):
    res=list()
    for img in imgs:
        img=Image.fromarray(img)
        img=img.resize(Image_size)
        img=np.array(img)
        res.append(img)
    res=np.stack(res)
    return res
def to_tensor(imgs):
    res = list()
    for img in imgs:
        img = Image.fromarray(img)
        img = ToTensor()(img)
        res.append(img)
    return torch.cat(res,dim=0)
    
def proc_state(state):
    s_tensor=[]
    for s in state:
        s_tensor.append(trans(s))
    s_tensor=torch.cat(s_tensor,dim=0)
    return s_tensor

In [30]:
img = Image.fromarray(state[0])

In [32]:
import numpy as np

In [20]:
def resample(state):
    return state[-24:]

### RUN

In [54]:
robot.perform_act('wait')
state = cam.get_state(camera,time_unit)
state = trans(resample(state))

In [55]:
while True:
    s = time.time()
    state_tensor = to_tensor(state)
    act_num = Q_Agent.get_action_eps(state_tensor)
    print 'action is %s'%actions[act_num]
    time.sleep(1)
    thread.start_new_thread(robot.perform_act,(actions[act_num],))
    time.sleep(0.3)
    n_state = cam.get_state(camera,time_unit)
    n_state = trans(resample(n_state))
    Q_Agent.memorize(state,act_num,n_state,ep,time_stamp)
    state = n_state
    time_stamp+=1
    print time.time()-s

action is call_people
4.31437897682
action is wave
4.31456708908
action is look
4.31544613838
action is look
4.32003998756
action is wave
4.31824398041
action is wave


KeyboardInterrupt: 

In [18]:
state.shape

(24, 480, 640)